In [1]:
#Importing the packages
import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import nltk
import re,string
nltk.download('stopwords')
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("C:\\Users\\Admin\\Downloads\\train.csv")

#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head(5)

C:\Users\Admin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


(159571, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
df["labels"] = df['toxic'].map({0: "Hate Speech", 1: "Offensive Speech"})
df = df[["comment_text", "labels"]]

In [4]:
 def preprocess(text):
        test_list = text.split()
        text = ""
        for word in test_list:
            if word.endswith("ing"):
                word=word[:-3]
            text+=word
            text+=" "
        doc = nlp(text)
        filtered_tokens = []
        
        for token in doc:
            if token.is_stop or token.is_punct:
                continue
            filtered_tokens.append(token.lemma_)
            
        return " ".join(filtered_tokens)

In [6]:
df["tweet_new"] = df.comment_text.apply(preprocess)

In [7]:
df.head()

,comment_text,labels,tweet_new
0,Explanation\r\nWhy the edits made under my use...,Hate Speech,Explanation edit username Hardcore Metallica F...
1,D'aww! He matches this background colour I'm s...,Hate Speech,D'aww match background colour seemingly stick ...
2,"Hey man, I'm really not trying to edit war. It...",Hate Speech,hey man try edit war guy constantly remov rele...
3,"""\r\nMore\r\nI can't make any real suggestions...",Hate Speech,real suggestion improvement wonder section sta...
4,"You, sir, are my hero. Any chance you remember...",Hate Speech,sir hero chance remember page


In [8]:
def clean(text):
    text = str (text). lower()
    text = re. sub('[.?]', '', text)
    text = re. sub('https?://\S+|www.\S+', '', text)
    text = re. sub('<.?>+', '', text)
    text = re. sub('[%s]' % re. escape(string. punctuation), '', text)
    text = re. sub('\n', '', text)
    text = re. sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ". join(text)
    text = [stemmer. stem(word) for word in text. split(' ')]
    text=" ". join(text)
    return text

In [9]:
df["tweet_new"] = df.tweet_new.apply(clean)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
x = np. array(df["tweet_new"])
y = np. array(df["labels"])
cv = TfidfVectorizer()
X = cv.fit_transform(x)
df.head()

,comment_text,labels,tweet_new
0,Explanation\r\nWhy the edits made under my use...,Hate Speech,explan edit usernam hardcor metallica fan reve...
1,D'aww! He matches this background colour I'm s...,Hate Speech,daww match background colour seem stick thank ...
2,"Hey man, I'm really not trying to edit war. It...",Hate Speech,hey man tri edit war guy constant remov relev ...
3,"""\r\nMore\r\nI can't make any real suggestions...",Hate Speech,real suggest improv wonder section statist lat...
4,"You, sir, are my hero. Any chance you remember...",Hate Speech,sir hero chanc rememb page


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = DecisionTreeClassifier()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "Hi sister"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.9413395620881521
hi sister
['Hate Speech']


In [11]:
from sklearn.naive_bayes import MultinomialNB

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = MultinomialNB()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "hi"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.7931287443452745
bastard
['Offensive Speech']


In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = RandomForestClassifier()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
y_pred#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "hi brother"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

In [26]:
from sklearn.svm import SVC

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#Model building
model = SVC()
#Training the model
model.fit(X_train,y_train)
#Testing the model
y_pred = model.predict (X_test)
#Accuracy Score of our model
from sklearn.metrics import accuracy_score
print (accuracy_score (y_test,y_pred))
#Predicting the outcome
inp = "hi"
inp = preprocess(inp)
print(inp)
inp = cv.transform([inp]).toarray()
print(model.predict(inp))

0.8991319232179973
hi
['No Hate and Offensive Speech']
